In [2]:
# USED FOR PARSING JSON:
import json
# USED TO ACCESS SODA (in Python 3.x, urllib2 is replaced with urllib.request):
import urllib.request
import pandas as pd

# THE API CALL:
apiURL = "https://opendata.camden.gov.uk/resource/ndrn-zq36.json?$select=name_1,local_type,district_borough,easting,northing,longitude,latitude"

# TRIGGER THE API CALL:
with urllib.request.urlopen(apiURL) as dataGet:
    # STORE RECORDS IN JSON OBJECT:
    dataResponse = json.load(dataGet)

# GET THE NUMBER OF RECORDS:
dataResults = len(dataResponse)


print(dataResults)

print(dataResponse[0])

1000
{'name_1': 'N17 8HF', 'local_type': 'Postcode', 'district_borough': 'Haringey', 'easting': '533094', 'northing': '191529', 'longitude': '-0.079418', 'latitude': '51.606877'}


In [3]:
# Convert to DataFrame
df = pd.DataFrame(dataResponse)

# Print the DataFrame
print(df.head())

print(df.columns)

    name_1 local_type     district_borough easting northing  longitude  \
0  N17 8HF   Postcode             Haringey  533094   191529  -0.079418   
1  EN4 9JT   Postcode               Barnet  527746   196073  -0.154945   
2  W1W 5DD   Postcode  City of Westminster  528861   182174  -0.143936   
3   N1 8SD   Postcode            Islington  532134   183998  -0.096101   
4  NW3 6EH   Postcode               Camden  526105   185013  -0.182624   

    latitude  
0  51.606877  
1  51.648951  
2  51.523791  
3  51.539427  
4  51.549927  
Index(['name_1', 'local_type', 'district_borough', 'easting', 'northing',
       'longitude', 'latitude'],
      dtype='object')
